# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-

In [75]:
!pip install chromadb langchain langchain_community langchain_openai unstructured beautifulsoup4


In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [76]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [77]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.agents import Tool, initialize_agent, AgentType

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [78]:
llm = OpenAI(temperature=0)

In [ ]:
# Path to the uploaded file :contentReference[oaicite:2]{index=2}
sonnets_path = r"C:\Users\happy\Documents\ironhack\Week17\set 2\lab-agent-vector-store\sonnets.txt"

# # Load the entire text as one Document
# loader = TextLoader(sonnets_path, encoding="utf-8")
# raw_docs = loader.load()   # ⇒ [Document(page_content=<all 154 sonnets>)]


In [ ]:
# raw_docs

[Document(metadata={'source': 'C:\\Users\\happy\\Documents\\ironhack\\Week17\\set 2\\lab-agent-vector-store\\sonnets.txt'}, page_content="\tSONNETS\n\n\n\nTO THE ONLY BEGETTER OF\nTHESE INSUING SONNETS\nMR. W. H. ALL HAPPINESS\nAND THAT ETERNITY\nPROMISED BY\nOUR EVER-LIVING POET WISHETH\nTHE WELL-WISHING\nADVENTURER IN\nSETTING FORTH\nT. T.\n\n\nI.\n\nFROM fairest creatures we desire increase,\nThat thereby beauty's rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou, contracted to thine own bright eyes,\nFeed'st thy light'st flame with self-substantial fuel,\nMaking a famine where abundance lies,\nThyself thy foe, to thy sweet self too cruel.\nThou that art now the world's fresh ornament\nAnd only herald to the gaudy spring,\nWithin thine own bud buriest thy content\nAnd, tender churl, makest waste in niggarding.\n  Pity the world, or else this glutton be,\n  To eat the world's due, by the grave and thee.\n\nII.\n\nWhen fo

In [ ]:
# from pathlib import Path

# relevant_parts = []
# for p in Path(".").absolute().parts:
#     relevant_parts.append(p)
#     if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
#         break
# doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [95]:
loader = TextLoader(sonnets_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="shakespeare_sonnets")

In [94]:
texts

[Document(metadata={'source': 'https://www.gutenberg.org/files/11/11-0.txt'}, page_content='*** START OF THE PROJECT GUTENBERG EBOOK 11 ***\n\n[Illustration]\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\nContents\n\nCHAPTER I.     Down the Rabbit-Hole CHAPTER II.    The Pool of Tears CHAPTER III.   A Caucus-Race and a Long Tale CHAPTER IV.    The Rabbit Sends in a Little Bill CHAPTER V.     Advice from a Caterpillar CHAPTER VI.    Pig and Pepper CHAPTER VII.   A Mad Tea-Party CHAPTER VIII.  The Queen’s Croquet-Ground CHAPTER IX.    The Mock Turtle’s Story CHAPTER X.     The Lobster Quadrille CHAPTER XI.    Who Stole the Tarts? CHAPTER XII.   Alice’s Evidence\n\nCHAPTER I.\n\nDown the Rabbit\n\nHole\n\nAlice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the us

In [96]:
sonnet_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a Shakespearean Sonnets assistant.  "
        "Use ONLY the following excerpts from Shakespeare’s Sonnets to answer the question.  \n"
        "If the answer cannot be found, reply “I don’t know.”\n\n"
        "CONTEXT:\n{context}\n\n"
        "QUESTION: {question}\n"
        "ANSWER:"
    ),
)

In [97]:
sonnet_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": sonnet_prompt},
)

In [98]:
print(sonnet_qa.run("Which Sonnet compares a summer's day to eternal lines?"))
print(sonnet_qa.run("In which Sonnet does Shakespeare mention 'eternity'?"))


 Sonnet XVIII.
 I don't know.


In [99]:
sonnets_tool = Tool(
    name="SonnetsQA",
    func=sonnet_qa.run,
    description="Answer questions about Shakespeare’s Sonnets.",
    return_direct=True
)


In [100]:
agent = initialize_agent(
    [sonnets_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [88]:
print(agent.run("What does Sonnet 18 say about mortality?"))
print(agent.run("Which sonnet addresses the theme of time's passage?"))




> Entering new AgentExecutor chain...
 I should use SonnetsQA to find the answer
Action: SonnetsQA
Action Input: Sonnet 18
Observation:  I don't know.


> Finished chain.
 I don't know.


> Entering new AgentExecutor chain...
 I should use SonnetsQA to find the answer.
Action: SonnetsQA
Action Input: "Which sonnet addresses the theme of time's passage?"
Observation:  Sonnet 60


> Finished chain.
 Sonnet 60


In [101]:
from langchain_community.document_loaders import WebBaseLoader

In [102]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [103]:
loader

In [ ]:
# %pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import pandas as pd
# from bs4 import BeautifulSoup
# from langchain.schema import Document

# # 1. Extract plain text from HTML
# plain_texts = []
# for doc in raw_docs:
#     soup = BeautifulSoup(doc.page_content, "html.parser")
#     text = soup.get_text(separator="\n")        # preserve paragraph breaks
#     plain_texts.append(text)

# # 2. Split into paragraphs and build DataFrame
# paras = []
# for text in plain_texts:
#     for p in text.split("\n\n"):                # split on double-newline
#         p = p.strip()
#         if p:
#             paras.append(p)

# df = pd.DataFrame({"text": paras})

# # 3. Normalize whitespace (collapse internal runs of spaces/newlines) 
# #    and drop any empty rows just in case:
# df["text"] = df["text"].str.replace(r"\s+", " ", regex=True).str.strip()
# df = df[df["text"] != ""]

# print(df.shape)  # now (n_paragraphs, 1)
# # print(df())
# df

In [ ]:
# from langchain.schema import Document
# from langchain_text_splitters import CharacterTextSplitter

# # wrap cleaned paragraphs back into Documents
# clean_docs = [Document(page_content=t) for t in df["text"]]

# splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = splitter.split_documents(clean_docs)

# now build your vector store on `texts`


In [104]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [51]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [105]:
tools = [
    Tool(
        name="SonnetsQA",
        func=sonnet_qa.run,
        description="Answer questions about Shakespeare’s Sonnets.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [106]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [107]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question since it is about a specific event and person.
Action: Ruff QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the State of the Union address?"
Observation:  I don't know.
Thought: I should try rephrasing the question to see if I can get a better answer.
Action: Ruff QA System
Action Input: "What was Biden's statement about Ketanji Brown Jackson during the State of the Union?"
Observation:  I don't know.
Thought: I should try using SonnetsQA since it may have more information on specific speeches and events.
Action: SonnetsQA
Action Input: "What did Biden say about Ketanji Brown Jackson in the State of the Union address?"
Observation:  I don't know.
Thought: I should try searching for news articles or transcripts of the State of the Union address to find the answer.
Action: Search for news articles/transcripts
Action Input: "Biden Ketanji Brown Jackson State of the U

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [108]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be some differences in their capabilities and features.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and automatic fixing of lint violations, while also being able to replace other tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. It also does not require the installation of Rust.
Thought: This information is helpful, but I should also consider other sources and do more research before making a decision.
Action: SonnetsQA
Action Input: "Why use ruff over flake8?"
Observation:  I don't know.
Thought: It seems like SonnetsQA is not the right tool for this question, as it is specifically designed for answering questions about Shakespeare's Sonnets.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule 

{'input': 'Why use ruff over flake8?',
 'output': 'Based on my research, it seems that ruff may be a more comprehensive and efficient option for python linting compared to flake8. However, the final decision should be based on individual needs and preferences.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [111]:
tools = [
    Tool(
        name="SonnetsQA",
        func=sonnet_qa.run,
        description="Answer questions about Shakespeare’s Sonnets.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [113]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [114]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should search for information about the state of the union address and any mentions of Ketanji Brown Jackson.
Action: SonnetsQA
Action Input: "What did biden say about ketanji brown jackson in the state of the union address?"
Observation:  I don't know.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " I don't know."}

In [115]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be some differences in their capabilities and features.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and automatic fixing of lint violations, while also being able to replace other tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. It also does not require the installation of Rust.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff offers a larger rule set and automatic fixing of lint violations, while also being able to replace other tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. It also does not require the installation of Rust.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [116]:
tools = [
    Tool(
        name="SonnetsQA",
        func=sonnet_qa.run,
        description="useful for when you need to answer questions about Shakespeare’s Sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [61]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [118]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks (notebook.codeActionsOnSave in VS Code).


> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': ' Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks (notebook.codeActionsOnSave in VS Code).'}

In [69]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(
    urls=["https://www.gutenberg.org/files/11/11-0.txt"],
    headers={"User-Agent": "MyVectorStoreBot/1.0 (+https://github.com/my-project)"}
)
my_docs = loader.load()
# 1) Load book
# my_docs = TextLoader("/path/to/alice_in_wonderland.txt").load()

# 2) Split into ~1 000-char chunks
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = splitter.split_documents(my_docs)

# 3) Embed & index
embeddings = OpenAIEmbeddings()
alice_db = Chroma.from_documents(texts, embeddings, collection_name="alice_in_wonderland")
my_docs

[Document(metadata={'source': 'https://www.gutenberg.org/files/11/11-0.txt'}, page_content='*** START OF THE PROJECT GUTENBERG EBOOK 11 ***\n\n[Illustration]\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\nContents\n\nCHAPTER I.     Down the Rabbit-Hole CHAPTER II.    The Pool of Tears CHAPTER III.   A Caucus-Race and a Long Tale CHAPTER IV.    The Rabbit Sends in a Little Bill CHAPTER V.     Advice from a Caterpillar CHAPTER VI.    Pig and Pepper CHAPTER VII.   A Mad Tea-Party CHAPTER VIII.  The Queen’s Croquet-Ground CHAPTER IX.    The Mock Turtle’s Story CHAPTER X.     The Lobster Quadrille CHAPTER XI.    Who Stole the Tarts? CHAPTER XII.   Alice’s Evidence\n\nCHAPTER I.\n\nDown the Rabbit\n\nHole\n\nAlice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the us

In [72]:
# 1. Wrap in RetrievalQA
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)
alice_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an assistant that answers questions about Alice’s Adventures in Wonderland.  "
        "Use ONLY the following excerpts from the text.  "
        "If the answer is not in the context, say “I don’t know.”\n\n"
        "CONTEXT:\n{context}\n\n"
        "QUESTION: {question}\n"
        "ANSWER:"
    ),
)

# Build the QA chain
alice_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=alice_db.as_retriever(),
    chain_type_kwargs={"prompt": alice_prompt},  # omit if you want default prompt
)


In [73]:
print(alice_qa.run("Who is the Cheshire Cat?"))
print(alice_qa.run("What does Alice say when she first meets the Caterpillar?"))


C:\Users\happy\AppData\Local\Temp\ipykernel_41472\2066584284.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(alice_qa.run("Who is the Cheshire Cat?"))


 The Cheshire Cat is a friend of Alice's.
 "So you think you're changed, do you?"


In [74]:
from langchain.agents import Tool, initialize_agent, AgentType

tools = [
    Tool(
        name="AliceBookQA",
        func=alice_qa.run,
        description="Answer questions about Alice’s Adventures in Wonderland",
        return_direct=True
    ),
    # …you could add more tools here…
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Now you can do:
print(agent.run("Tell me what happens in the Mad Tea Party chapter."))




> Entering new AgentExecutor chain...
 I should use AliceBookQA to answer this question.
Action: AliceBookQA
Action Input: "What happens in the Mad Tea Party chapter?"
Observation:  Alice comes across the house of the March Hare and the Hatter, who are having a tea party with a sleeping Dormouse. The Cat's head disappears and the King and executioner search for it while the rest of the party continues with their game. The Duchess and Alice walk off together and Alice is relieved to find her in a pleasant mood.


> Finished chain.
 Alice comes across the house of the March Hare and the Hatter, who are having a tea party with a sleeping Dormouse. The Cat's head disappears and the King and executioner search for it while the rest of the party continues with their game. The Duchess and Alice walk off together and Alice is relieved to find her in a pleasant mood.
